# training the binary classification task using a VGG16 network written from scratch

* The python file model.py contains the code for the custom written VGG16 network
* this notebook also performs training on a pretrained ResNet50 netwoek which is much more sophisticated network compared to VGG16.  * results of both models will be evalusted in the evaluation step
* in addition to the model, a train function is also written (in model.py) and imported here which is basically wrapper that performs the forward and backward loop of the train and forward loop of the eval model simultaneously.
* the function "get_loader" is a custom wrapper that makes a pytorch dataset and applies a dataloader to it.

In [4]:
import os
os.chdir('/Users/simpleai/Desktop/az_task/scripts/')
import pandas as pd
from torchvision import transforms
import torch
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights
import warnings
warnings.filterwarnings('ignore')

from dataset import get_loader # custom written for this task
from model import MyVGG16      # custom written for this task
from train import train        # custom written for this task

In [24]:
# hyperparameters
NUM_CLASSES = 2
NUM_EPOCHS = 40
BATCH_SIZE = 4
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 1E-4
CLASS_WEIGHT = torch.tensor([0.8,0.2])
device = "mps" if torch.backends.mps.is_available() else "cpu" # I'm using an M1 chip which can perform vectorized calculations over Mac GPU using ARM technology

In [27]:
# defining the transforms to be applied to the images
# transform will include on-the-fly augmentations later
transform = transforms.Compose([
   transforms.ToTensor(), 
   transforms.Resize((256,256)), # resizing to 256 by 256 due to having limited amount of ram available on my Mac
   transforms.Normalize(
       mean=[0.4914, 0.4822, 0.4465], 
       std=[0.2023, 0.1994, 0.2010]
       )
   ])

# opening the label_df where both labels and path to images are located in
label_df_path = '/Users/simpleai/Desktop/az_task/results/task1/binary_classification_metadata.csv'
label_df = pd.read_csv(label_df_path, index_col=0)

# dividing the label_df into train, val, test dataset
train_df = label_df.loc[label_df['type']=='Train', :] #1519 Disease; 401 Healthy 0.8:0.2 imbalance
#train_df = train_df.sample(frac=0.25) # selecting a random quarter of the data for the speed
train_df = train_df.reset_index(drop=True)

val_df = label_df.loc[label_df['type']=='Val', :]
#val_df = val_df.sample(frac=0.25) # selecting a random quarter of the data for the speed
val_df = val_df.reset_index(drop=True)

test_df = label_df.loc[label_df['type']=='Test', :]
test_df = test_df.reset_index(drop=True)

# defining the dataloader dictionary
dataloader_dict = {
    'train': get_loader(
        data_df = train_df, 
        label_cols = ['Healthy', 'Disease_Risk'],
        batch_size = BATCH_SIZE, 
        transform = transform, 
        shuffle = True),
    'val': get_loader(
        data_df = val_df, 
        label_cols = ['Healthy', 'Disease_Risk'],
        batch_size = BATCH_SIZE, 
        transform = transform, 
        shuffle = False),
    }

### Traing VGG16 model

In [28]:
# defining the model
model = MyVGG16(num_classes=NUM_CLASSES, input_height=256, input_width=256).to(device)


# defining criterion, optimizer
criterion = nn.CrossEntropyLoss(weight=CLASS_WEIGHT).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# training the model
model, train_val_df = train(
    model, 
    dataloader_dict, 
    criterion, 
    optimizer, 
    device, 
    checkpoint_path='/Users/simpleai/Desktop/az_task/results/task2/vgg16_binary_classifier.pth', 
    train_val_info_path='/Users/simpleai/Desktop/az_task/results/task2/vgg16_binary_classifier_info.csv',
    num_epochs=NUM_EPOCHS,
    vgg=True
    )


Epoch 1/40
__________


100%|█████████████████████████████████████████| 480/480 [06:16<00:00,  1.28it/s]


epoch 1 train >>> loss: 0.264, accuracy: 0.620


100%|█████████████████████████████████████████| 160/160 [01:04<00:00,  2.49it/s]


epoch 1 val >>> loss: 0.165, accuracy: 0.786
saving the best model in /Users/simpleai/Desktop/az_task/results/task2/vgg16_binary_classifier.pth
Epoch 2/40
__________


...

epoch 5 train >>> loss: 0.158, accuracy: 0.732


100%|█████████████████████████████████████████| 160/160 [01:03<00:00,  2.52it/s]


epoch 5 val >>> loss: 0.154, accuracy: 0.631
Epoch 6/40
__________


 31%|████████████▉                            | 151/480 [01:51<04:03,  1.35it/s]


KeyboardInterrupt: 

### Training pretrained ResNet50 model

In [30]:
# Using pretrained weights:
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

# modifying output number of classes from 1000 to NUM_CLASSES
model.fc = nn.Linear(2048, NUM_CLASSES)
model = model.to(device)

# defining criterion, optimizer
criterion = nn.CrossEntropyLoss(weight=CLASS_WEIGHT).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# training the model
model, train_val_df = train(
    model, 
    dataloader_dict, 
    criterion, 
    optimizer, 
    device, 
    checkpoint_path='/Users/simpleai/Desktop/az_task/results/task2/resnet50_binary_classifier.pth', 
    train_val_info_path='/Users/simpleai/Desktop/az_task/results/task2/resnet50_binary_classifier_info.csv',
    num_epochs=NUM_EPOCHS
    )

Epoch 1/40
__________


100%|█████████████████████████████████████████| 480/480 [03:47<00:00,  2.11it/s]


epoch 1 train >>> loss: 0.132, accuracy: 0.778


100%|█████████████████████████████████████████| 160/160 [00:56<00:00,  2.85it/s]


epoch 1 val >>> loss: 0.086, accuracy: 0.842
saving the best model in /Users/simpleai/Desktop/az_task/results/task2/resnet50_binary_classifier.pth
Epoch 2/40
__________


...

epoch 39 val >>> loss: 0.151, accuracy: 0.825
Epoch 40/40
__________


100%|█████████████████████████████████████████| 480/480 [04:12<00:00,  1.90it/s]


epoch 40 train >>> loss: 0.001, accuracy: 0.999


100%|█████████████████████████████████████████| 160/160 [01:05<00:00,  2.45it/s]

epoch 40 val >>> loss: 0.177, accuracy: 0.836
